# Requesting data from the NCBI test dataset
The data requested is available here: https://huggingface.co/datasets/ncbi_disease

In [1]:
import requests
import pandas as pd 

Followed the instructions from here:
https://huggingface.co/docs/datasets-server/quick_start

In [2]:
API_URL = "https://datasets-server.huggingface.co/rows?dataset=ncbi_disease&config=ncbi_disease&split=test&offset="

In [3]:
# You can only get 100 rows at once
def get100RowsOfData(offset):
    response = requests.get(API_URL+str(offset)+"&length=100")
    return response.json()

In [4]:
# Gathering all of the tags and tokens
all_tokens = []
all_tags = []
all_ids = []

def getTokensAndTags(data):
    for row in data["rows"]:
        all_tokens.append(row["row"]["tokens"])
        ner_tags = row["row"]["ner_tags"]
        for i in range(len(ner_tags)):
            # Converting 0, 1 and 2 to IOB format
            if ner_tags[i] == 0:
                ner_tags[i] = 'O'
            elif ner_tags[i] == 1:
                ner_tags[i] = 'B-DIS'
            elif ner_tags[i] == 2:
                ner_tags[i] = 'I-DIS'
        all_tags.append(ner_tags)
        all_ids.append(row["row"]["id"])

In [5]:
# Requesting data by 100 rows (in total 940 rows)
current_offset = 0
for i in range(10):
    current_data = get100RowsOfData(current_offset)
    current_offset += 100
    getTokensAndTags(current_data)

In [14]:
# Writing data to csv file
data = {"id":all_ids,
       "tokens":all_tokens,
       "original_tags": all_tags}
df = pd.DataFrame(data)
df.to_csv("ncbi_test.csv")